In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('film_reviews_8250_original.csv',  sep='\t')

In [3]:
pd.set_option('display.max_rows',753)

In [4]:
#good_review = 'interesting fascinating inspirational'
#bad_review = 'boring disgusting uninteresting tedious'
#good_review = ''
#bad_review = ''

In [5]:
'''nan_array = list(pd.isna(data['review']))
for i in range(len(nan_array)):
    if nan_array[i]:
        if data.quality[i] == 1:
            data.review[i] = good_review
        else:
            data.review[i] = bad_review'''

"nan_array = list(pd.isna(data['review']))\nfor i in range(len(nan_array)):\n    if nan_array[i]:\n        if data.quality[i] == 1:\n            data.review[i] = good_review\n        else:\n            data.review[i] = bad_review"

In [6]:
for i in range(data.shape[0]):
    if data.raiting[i] >=7:
        data.quality[i] = 1
    else:
        data.quality[i] = 0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
data.index[6]

6

In [8]:
arr = []
nan_array = list(pd.isna(data['review']))
for i in range(data.shape[0]):
    if data.raiting[i] < 7 and data.raiting[i] > 6:
        arr.append(i)
    if i not in arr and nan_array[i]:
        arr.append(i)

In [9]:
data = data.drop(arr)

In [10]:
data

,name,review,raiting,quality
0,Toy Story (1995),as the favourite characters for adults.Overall...,8.3,1
3,Waiting to Exhale (1995),"unfeeling creatures. So basically, I feel that...",5.8,0
4,Father of the Bride Part II (1995),I don't think George Banks could possibly have...,6.0,0
5,Heat (1995),want to lead. Pacino's cop is less easy to sym...,8.2,1
7,Tom and Huck (1995),scenes. The graveyard scene is well done. Inju...,5.6,0
8,Sudden Death (1995),"the gunplay and the action during the game), a...",5.7,0
9,GoldenEye (1995),"produced by Eon Productions, and the first to ...",7.2,1
11,Dracula: Dead and Loving It (1995),engaged couple who after 10 years have suddenl...,5.8,0
12,Balto (1995),of the movie. It especially sounds great when ...,7.1,1
13,Nixon (1995),ready for the day when they turned on him. It...,7.1,1


In [11]:
len(set(data.name))

5346

In [12]:
len(data.name)

5351

In [13]:
#data = data.dropna()

In [14]:
data.shape

(5351, 4)

In [15]:
#for i in range(data.shape[0]):
    #print(i)
    #data['review'][i] = 0 if data['raiting'][i] < 7 else 1

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
import string
exclude = set(string.punctuation)

In [18]:
data['review'] = data['review'].apply(lambda x:
                                      "".join(i for i in x if i not in exclude))

In [19]:
#data.to_csv('film_reviews_8250_changed.csv', sep='\t', index=False)

In [20]:
data['review'] = data['review'].apply(lambda x:
                                      " ".join(lemmatizer.lemmatize(i) for i in x.split()))

In [21]:
title = "sfsfd sdf sf sfd"

In [22]:
" ".join(lemmatizer.lemmatize(i) for i in title.split())


'sfsfd sdf sf sfd'

In [23]:
vectorizer = CountVectorizer().fit(data['review'])
features = vectorizer.transform(data['review'])

In [24]:
clf = LogisticRegression()
clf.fit(features[:data.shape[0]-400], data.quality[:data.shape[0]-400])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
pd.set_option('display.max_rows',753)

In [26]:
data

,name,review,raiting,quality
0,Toy Story (1995),a the favourite character for adultsOverall th...,8.3,1
3,Waiting to Exhale (1995),unfeeling creature So basically I feel that th...,5.8,0
4,Father of the Bride Part II (1995),I dont think George Banks could possibly have ...,6.0,0
5,Heat (1995),want to lead Pacinos cop is le easy to sympath...,8.2,1
7,Tom and Huck (1995),scene The graveyard scene is well done Injun J...,5.6,0
8,Sudden Death (1995),the gunplay and the action during the game and...,5.7,0
9,GoldenEye (1995),produced by Eon Productions and the first to s...,7.2,1
11,Dracula: Dead and Loving It (1995),engaged couple who after 10 year have suddenly...,5.8,0
12,Balto (1995),of the movie It especially sound great when yo...,7.1,1
13,Nixon (1995),ready for the day when they turned on him It g...,7.1,1


In [27]:
data.shape

(5351, 4)

In [28]:
review = 

SyntaxError: invalid syntax (<ipython-input-28-81541640e313>, line 1)

In [ ]:
#МЕТРИКА

In [29]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [30]:
y_true = np.array(list(data.quality[data.shape[0]-399:]))#метки
y_pred = np.array(list(clf.predict(features[data.shape[0]-399:])))#предсказания
t = 0.4

In [31]:
accuracy_score(y_true, y_pred, t)

0.8671679197994987

In [32]:
recall_score(y_true, y_pred, t)

0.8740157480314961

In [33]:
precision_score(y_true, y_pred, t)

0.9135802469135802

In [35]:
diction_ = [ (k, v) for k, v in vectorizer.vocabulary_.items()]
diction_.sort(key=lambda x: x[1])
diction_ = pd.DataFrame(diction_, columns=['Feature', 'Position'])
diction_.head()

,Feature,Position
0,000,0
1,001,1
2,007,2
3,00s,3
4,010,4


In [36]:
diction_['Weight'] = clf.coef_[0]

In [37]:
diction_.sort_values('Weight')

,Feature,Position,Weight
4836,bad,4836,-1.108692
60961,worst,60961,-0.922153
20904,flick,20904,-0.839202
13917,decent,13917,-0.837610
17858,enjoyable,17858,-0.816637
47617,scenery,47617,-0.793864
811,610,811,-0.766047
60939,worse,60939,-0.754087
871,710,871,-0.741540
13213,cute,13213,-0.740863


In [26]:
s= 'sd&f'

In [27]:
s.replace('&', ' ')

'sd f'